In [ ]:
import keras
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from keras.layers import Dense, Dropout, Input, Conv2D, Flatten, BatchNormalization, Reshape
from keras.models import Model,Sequential
from keras.datasets import fashion_mnist
from tqdm import tqdm
from keras.layers.advanced_activations import LeakyReLU
(x_train,y_train), (x_test, y_test) = fashion_mnist.load_data()
x_train = (x_train.astype(np.float32) - 127.5)/127.5

Generator

In [ ]:
def create_generator():
    generator=Sequential()
    generator.add(Conv2D(64, (3,3), strides =(3,3), padding ='same', input_shape = [28,28,1]))
    generator.add(BatchNormalization())
    generator.add(LeakyReLU(0.2))
    
    generator.add(Conv2D(32, (3,3), strides =(3,3), padding ='same'))
    generator.add(BatchNormalization())
    generator.add(LeakyReLU(0.2))
  
    
    generator.add(Flatten())
    generator.add(Dense(units=512))
    generator.add(LeakyReLU(0.2))
    
    generator.add(Dense(units=1024))
    generator.add(LeakyReLU(0.2))
    
    generator.add(Dense(units=28*28, activation='tanh'))
    #generator.add(Reshape((28,28)))
    opt = keras.optimizers.Adam(learning_rate=0.1)
    generator.compile(loss='binary_crossentropy', optimizer=opt)
    return generator
g=create_generator()
g.summary()

Model: "sequential_13"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_14 (Conv2D)           (None, 10, 10, 64)        640       
_________________________________________________________________
batch_normalization_14 (Batc (None, 10, 10, 64)        256       
_________________________________________________________________
leaky_re_lu_46 (LeakyReLU)   (None, 10, 10, 64)        0         
_________________________________________________________________
conv2d_15 (Conv2D)           (None, 4, 4, 32)          18464     
_________________________________________________________________
batch_normalization_15 (Batc (None, 4, 4, 32)          128       
_________________________________________________________________
leaky_re_lu_47 (LeakyReLU)   (None, 4, 4, 32)          0         
_________________________________________________________________
flatten_7 (Flatten)          (None, 512)             

Discriminator

In [ ]:
def create_discriminator():
    discriminator=Sequential()
    discriminator.add(Dense(units=512,input_dim=28*28))
    #discriminator.add(Conv2D(64, (3,3), strides =(1,1), padding ='same', input_shape = [28,28,1]))
    discriminator.add(LeakyReLU(0.2))
    discriminator.add(Dropout(0.3))
    #discriminator.add(Flatten())
       
    
    discriminator.add(Dense(units=512))
    discriminator.add(LeakyReLU(0.2))
    discriminator.add(Dropout(0.3))
       
    discriminator.add(Dense(units=256))
    discriminator.add(LeakyReLU(0.2))
    
    discriminator.add(Dense(units=1, activation='sigmoid'))
    opt = keras.optimizers.Adam(learning_rate=0.1)
    discriminator.compile(loss='binary_crossentropy', optimizer=opt)
    return discriminator
d =create_discriminator()
d.summary()

Model: "sequential_14"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_48 (Dense)             (None, 512)               401920    
_________________________________________________________________
leaky_re_lu_50 (LeakyReLU)   (None, 512)               0         
_________________________________________________________________
dropout_12 (Dropout)         (None, 512)               0         
_________________________________________________________________
dense_49 (Dense)             (None, 512)               262656    
_________________________________________________________________
leaky_re_lu_51 (LeakyReLU)   (None, 512)               0         
_________________________________________________________________
dropout_13 (Dropout)         (None, 512)               0         
_________________________________________________________________
dense_50 (Dense)             (None, 256)             

In [ ]:
def create_gan(discriminator, generator):
     discriminator.trainable=False
     img_shape=(28, 28, 1)
     gan_input = Input(img_shape)
     x = generator(gan_input)
     gan_output= discriminator(x)
     gan= Model(inputs=gan_input, outputs=gan_output)
     opt = keras.optimizers.Adam(learning_rate=0.1)
     gan.compile(loss='binary_crossentropy',    optimizer=opt)
     return gan

In [ ]:
generator= create_generator()
discriminator= create_discriminator()
gan = create_gan(discriminator, generator)

batch_size = 128
Epochs = 200
for epcs in range(1, Epochs ):
        for _ in tqdm(range(batch_size)):
            noise= np.random.normal(0,1, [batch_size, 28,28,1])
            generated_images = generator.predict(noise)
            image_batch =x_train[np.random.randint(low=0,high=x_train.shape[0],size=128)]
            image_batch = image_batch.reshape(-1,28*28)
            X= np.concatenate([image_batch, generated_images])
            y_discriminator=np.zeros(2*batch_size)
            y_discriminator[:batch_size]=1
            discriminator.trainable=True
            discriminator.train_on_batch(X, y_discriminator)
            noise= np.random.normal(0,1, [batch_size, 28,28,1])
            y_label = np.ones(batch_size)
            discriminator.trainable=False
            gan.train_on_batch(noise, y_label)

 94%|█████████▍| 120/128 [00:22<00:01,  5.30it/s]

In [ ]:
generated_images = generator.predict(np.random.normal(0,1,[batch_size, 28,28,1]))
x =generated_images[0].reshape(28,28)
plt.imshow(x, cmap ="Greys")
